In [5]:
import pandas as pd
import requests

In [6]:
# 1. Event detection

In [15]:
# Read gaze data with AOIs using pandas library
data = pd.read_csv("data/gazeDataWithAOIs.csv")
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

c:\users\eye-tracking\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3194: DtypeWarning: Columns (3,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# Preview data
data.head()

In [16]:
#Keep only the gaze data coming from the eye-tracking device
data = data[ data['EventSource.1'] == 1 ]

In [17]:
#Fill n/a values
data = data.fillna('')

In [18]:
#Event detection configuration
config = {
                "fixationFilter" : "IVT",
                "screenResolutionWidth" : 1920, 
                "screenResolutionHeight" : 1080, 
                "monitorSize" : 17, 
                "screenDistance" : 70,
                "gapFill" : False, 
                "maxGapLength": 75, 
                "noiseReduction" : True, 
                "filterType" : "Median", 
                "windowNoise" : 3,
                "windowVelocity" : 20, 
                "velocityThreshold" : 30, 
                "mergeFixation" : True, 
                "maxTimeBtwFixation" : 75,
                "maxAngleBtwFixation" : 0.5, 
                "discardShortFixation" : True,
                "minDurationFixation" : 60
}

In [19]:
# POST gaze data to the (R) eventDetection server
# url = 'http://rserver:6789/eventDetection'
url = 'http://127.0.0.1:6789/eventDetection'
payload = {'config': config, 'data': data.to_dict(orient="records")}
response = requests.post(url, json = payload)

In [20]:
# Create new a dataframe from the json response of the (R) eventDetection server
dataWithEvents = pd.DataFrame.from_records(response.json())

In [21]:
# Preview gaze data with event information
dataWithEvents.head()

,Row,Timestamp,EventSource,SlideEvent,StimType,Duration,CollectionPhase,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,128.9448,,,,,,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,137.2724,,,,,,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,500.0,567.9048,16.1935,1.0,,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,145.6203,,,,,,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,1,0,0,0,0,0,1,0,0,0,0,0,948.0,500.0,566.2654,4.7572,1.0,,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,2.6143,-1369.9611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,153.9470,,,,,,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,567.7859,27.6866,1.0,,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,2753.7102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,162.2703,,,,,,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,509.0,567.7856,27.6456,1.0,,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,-4.9222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# export dataWithEvents with events as csv
dataWithEvents.to_csv("data/gazeDataWithAOIsAndEvents.csv",  index=False)